## 0. 학습 세팅

### 1) 메모리 정리

In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

### 2) 수정된 코드 자동 업데이트

In [2]:
%load_ext autoreload
%autoreload 2
import foolbox as fb

## 1. Load library

In [3]:
# 라이브러리 호출
import os
import time
from collections import Counter, defaultdict

import argparse
import easydict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchmetrics.functional.classification import accuracy
from torch.utils.tensorboard import SummaryWriter

from src.engines import train, evaluate, epoch_time
from src.utils import load_checkpoint, save_checkpoint

## 2. Variable Declaration

In [4]:
# # Jupyter 외 환경
# parser = argparse.ArgumentParser()
# parser.add_argument("--title", type=str, default="baseline")
# parser.add_argument("--device", type=str, default="cuda")
# parser.add_argument("--root", type=str, default="data")
# parser.add_argument("--batch_size", type=int, default=64)
# parser.add_argument("--num_workers", type=int, default=2)
# parser.add_argument("--epochs", type=int, default=100)
# parser.add_argument("--lr", type=float, default=0.001)
# parser.add_argument("--logs", type=str, default='logs')
# parser.add_argument("--checkpoints", type=str, default='checkpoints')
# parser.add_argument("--resume", type=bool, default=False)
# args = parser.parse_args()

# Jupyter 환경
args = easydict.EasyDict({
        "title" : "VEEM_Test",
        "learn_type" : "classification",
        "device" : "cuda",
        "batch_size" : 32, # !!!
        "num_workers" : 2,
        "epochs" : 50, # !!!### 2) 모델 + 옵티마이저 + 손실함수 + 스케쥴러 + 메트릭 함수 정의
        "lr" : 0.01, # !!!
        "logs" : "logs",
        "checkpoints" : "checkpoints",
        "resume" : False,
        "test_ratio" : 0.25,
        "input_size" : 11,
        "hidden_size" : 2,
        "num_layers" : 1,
        "output_size" : 5,
        "regression_output_size" : 5,
        "classification_output_size" : 2,
        "regression_lr" : 0.01,
        "classification_lr" : 3e-3,
        "regression_epochs" : 50,
        "classification_epochs" : 100
    })

if(args.learn_type == "regression"):
    args.output_size = args.regression_output_size
    args.lr = args.regression_lr
    args.epochs = args.regression_epochs
else:
    args.output_size = args.classification_output_size
    args.lr = args.classification_lr
    args.epochs = args.classification_epochs

"""
print(args.output_size)
"""

'\nprint(args.output_size)\n'

## 3. Model Define

### 1) 모델 정의

In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x): # x : [batch_size, sequence_length, input_size]
        """
        print("x.shape : ", x.shape)
        print("x : ", x)
        """
        # hidden state + cell state 초기화 (Bi-directional LSTM : 아래의 hidden and cell states의 첫번째 차원은 2*self.num_layers)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(args.device) # h0 : [num_layers, batch_size, hidden_size]
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(args.device) # c0 : [num_layers, batch_size, hidden_size]
        """
        print("h0.shape : ", h0.shape)
        print("c0.shape : ", c0.shape)
        
        print("h0 : ", h0)
        print("c0 : ", c0)
        """

        # LSTM 순전파
        out, (hn, cn) = self.lstm(x, (h0, c0)) # out : [batch_size, sequence_length, hidden_size]
        """
        print("out.shape : ", out.shape)
        print("out : ", out)
        """
        
        # 마지막 time step(sequence length)의 hidden state 반환
        """
        print("out[:, -1, :] : ", out[:, -1, :])
        """
        out = self.fc(out[:, -1, :]) # out : [batch_size, hidden_size] -> out : [batch_size, output_size]
        """
        print("out.shape : ", out.shape)
        print("out : ", out)
        """
        return out

### 2) 모델 + 옵티마이저 + 손실함수 + 스케쥴러 + 메트릭 함수 정의

In [6]:
# Build model
model = RNN(args.input_size, args.hidden_size, args.num_layers, args.output_size).to(args.device)

# Build optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

# Build scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs  * 37)

# Build loss function + Build metric function
if(args.learn_type == "classification"):
    loss_fn = nn.CrossEntropyLoss()
    metric_fn = accuracy
else:
    loss_fn = nn.MSELoss()
    metric_fn = None

### 3) loggger 정의

In [7]:
# Build logger
train_logger = SummaryWriter(f'{args.logs}/train/{args.title}')
test_logger = SummaryWriter(f'{args.logs}/test/{args.title}')

## 4. Data Preprocessing

## - Person, SNSB

### 1) 데이터셋 변환

### 2) 데이터셋 불러오기

In [8]:
# 데이터셋 불러오기 + 출력
Person_path = f'C:\\Users\\Bang\\JupyterProjects\\VEEM_Project\\data\\Person_SNSB\\Person.csv'
SNSB_path = f'C:\\Users\\Bang\\JupyterProjects\\VEEM_Project\\data\\Person_SNSB\\SNSB.csv'

Person_dataset=pd.read_csv(Person_path)
SNSB_dataset=pd.read_csv(SNSB_path)

print("ㅡㅡㅡㅡㅡ[Person_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(Person_dataset.dtypes)
print("ㅡㅡㅡㅡㅡ[SNSB_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(SNSB_dataset.dtypes)

ㅡㅡㅡㅡㅡ[Person_dataset.dtypes]ㅡㅡㅡㅡㅡ
번호        int64
성명 코드    object
집단       object
성별       object
나이        int64
교육연한      int64
dtype: object
ㅡㅡㅡㅡㅡ[SNSB_dataset.dtypes]ㅡㅡㅡㅡㅡ
번호                      int64
성명 코드                  object
집단                     object
DST_F+B                 int64
S-K-BNT                 int64
RCFT_copyscore        float64
SVLT_delayedrecall      int64
K-TMT-E_B               int64
dtype: object


### 3) 데이터 자료형 변환

In [9]:
Person_dataset['나이'] = Person_dataset['나이'].astype(float)
Person_dataset['교육연한'] = Person_dataset['교육연한'].astype(float)

SNSB_dataset['DST_F+B'] = SNSB_dataset['DST_F+B'].astype(float)
SNSB_dataset['S-K-BNT'] = SNSB_dataset['S-K-BNT'].astype(float)
SNSB_dataset['SVLT_delayedrecall'] = SNSB_dataset['SVLT_delayedrecall'].astype(float)
SNSB_dataset['K-TMT-E_B'] = SNSB_dataset['K-TMT-E_B'].astype(float)

print("ㅡㅡㅡㅡㅡ[Person_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(Person_dataset.dtypes)
print("ㅡㅡㅡㅡㅡ[SNSB_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(SNSB_dataset.dtypes)

ㅡㅡㅡㅡㅡ[Person_dataset.dtypes]ㅡㅡㅡㅡㅡ
번호         int64
성명 코드     object
집단        object
성별        object
나이       float64
교육연한     float64
dtype: object
ㅡㅡㅡㅡㅡ[SNSB_dataset.dtypes]ㅡㅡㅡㅡㅡ
번호                      int64
성명 코드                  object
집단                     object
DST_F+B               float64
S-K-BNT               float64
RCFT_copyscore        float64
SVLT_delayedrecall    float64
K-TMT-E_B             float64
dtype: object


### 4) 인덱스 지정

In [10]:
Person_dataset.set_index('번호', inplace=True)
SNSB_dataset.set_index('번호', inplace=True)

In [11]:
print("ㅡㅡㅡㅡㅡ[Person_dataset]ㅡㅡㅡㅡㅡ")
Person_dataset

ㅡㅡㅡㅡㅡ[Person_dataset]ㅡㅡㅡㅡㅡ


,성명 코드,집단,성별,나이,교육연한
번호,,,,,
2,NSU1,HC,여성,71.0,16.0
3,CSR1,HC,남성,73.0,16.0
4,KMO1,MCI,여성,59.0,12.0
6,LCH1,MCI,남성,83.0,14.0
7,LYH1,MCI,여성,67.0,15.0
8,MYG1,HC,남성,60.0,16.0
9,YHR1,HC,남성,79.0,16.0
10,LJG1,MCI,남성,69.0,12.0
11,PHG1,MCI,남성,86.0,12.0


In [12]:
print("ㅡㅡㅡㅡㅡ[SNSB_dataset]ㅡㅡㅡㅡㅡ")
SNSB_dataset

ㅡㅡㅡㅡㅡ[SNSB_dataset]ㅡㅡㅡㅡㅡ


,성명 코드,집단,DST_F+B,S-K-BNT,RCFT_copyscore,SVLT_delayedrecall,K-TMT-E_B
번호,,,,,,,
2,NSU1,HC,13.0,11.0,35.0,11.0,16.0
3,CSR1,HC,10.0,15.0,30.0,5.0,26.0
4,KMO1,MCI,4.0,11.0,35.0,1.0,50.0
6,LCH1,MCI,5.0,8.0,18.5,0.0,300.0
7,LYH1,MCI,12.0,10.0,36.0,7.0,36.0
8,MYG1,HC,11.0,13.0,36.0,6.0,20.0
9,YHR1,HC,11.0,13.0,34.0,7.0,32.0
10,LJG1,MCI,9.0,12.0,27.0,0.0,43.0
11,PHG1,MCI,8.0,9.0,28.0,0.0,90.0


### 5) 필요한 피쳐 추출

In [13]:
# 데이터셋 나누기
SNSB_all_dataset=SNSB_dataset.iloc[:, 2:]
Person_all_dataset=Person_dataset.iloc[:, 2:]
Person_dementia_dataset=Person_dataset.iloc[:, 1:2]

In [14]:
print("ㅡㅡㅡㅡㅡ[SNSB_all_dataset]ㅡㅡㅡㅡㅡ")
SNSB_all_dataset

ㅡㅡㅡㅡㅡ[SNSB_all_dataset]ㅡㅡㅡㅡㅡ


,DST_F+B,S-K-BNT,RCFT_copyscore,SVLT_delayedrecall,K-TMT-E_B
번호,,,,,
2,13.0,11.0,35.0,11.0,16.0
3,10.0,15.0,30.0,5.0,26.0
4,4.0,11.0,35.0,1.0,50.0
6,5.0,8.0,18.5,0.0,300.0
7,12.0,10.0,36.0,7.0,36.0
8,11.0,13.0,36.0,6.0,20.0
9,11.0,13.0,34.0,7.0,32.0
10,9.0,12.0,27.0,0.0,43.0
11,8.0,9.0,28.0,0.0,90.0


In [15]:
print("ㅡㅡㅡㅡㅡ[Person_all_dataset]ㅡㅡㅡㅡㅡ")
Person_all_dataset

ㅡㅡㅡㅡㅡ[Person_all_dataset]ㅡㅡㅡㅡㅡ


,성별,나이,교육연한
번호,,,
2,여성,71.0,16.0
3,남성,73.0,16.0
4,여성,59.0,12.0
6,남성,83.0,14.0
7,여성,67.0,15.0
8,남성,60.0,16.0
9,남성,79.0,16.0
10,남성,69.0,12.0
11,남성,86.0,12.0


In [16]:
print("ㅡㅡㅡㅡㅡ[Person_dementia_dataset]ㅡㅡㅡㅡㅡ")
Person_dementia_dataset

ㅡㅡㅡㅡㅡ[Person_dementia_dataset]ㅡㅡㅡㅡㅡ


,집단
번호,
2,HC
3,HC
4,MCI
6,MCI
7,MCI
8,HC
9,HC
10,MCI
11,MCI


### 6) 데이터 변환

#### (1) Person_dementia_dataset : pandas -> list -> 문자열 임베딩 -> torch 변환

In [17]:
# for idx in Person_dementia_dataset.index:
#     print(Person_dementia_dataset.loc[idx,'집단'])

# pandas -> list
Person_dementia_dataset_list = Person_dementia_dataset['집단'].values.tolist()
print("Person_dementia_dataset_list : ", Person_dementia_dataset_list)

# list -> 문자열 임베딩
embedding_table = {'HC': 0, 'MCI': 1}
Person_dementia_dataset_embedding_list = []
for i, word in enumerate(Person_dementia_dataset_list):
    Person_dementia_dataset_embedding_list.append(embedding_table[word])
print("Person_dementia_dataset_embedding_list : ", Person_dementia_dataset_embedding_list)

# list -> torch 변환
Person_dementia_dataset_torch = torch.Tensor(Person_dementia_dataset_embedding_list)
print("Person_dementia_dataset_torch : ", Person_dementia_dataset_torch)

Person_dementia_dataset_list :  ['HC', 'HC', 'MCI', 'MCI', 'MCI', 'HC', 'HC', 'MCI', 'MCI', 'MCI', 'HC', 'HC', 'HC', 'HC', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'HC', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'HC', 'HC', 'HC', 'MCI', 'MCI', 'HC', 'HC', 'MCI', 'HC']
Person_dementia_dataset_embedding_list :  [0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0]
Person_dementia_dataset_torch :  tensor([0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1.,
        0.])


#### (2) SNSB_all_dataset : pandas -> numpy -> torch 변환

In [18]:
SNSB_all_dataset_np = SNSB_all_dataset.to_numpy()
SNSB_all_dataset_torch = torch.from_numpy(SNSB_all_dataset_np).float()
"""
print("ㅡㅡㅡㅡㅡ[SNSB_all_dataset_torch.shape]ㅡㅡㅡㅡㅡ")
print(SNSB_all_dataset_torch.shape)

print("ㅡㅡㅡㅡㅡ[SNSB_all_dataset_torch]ㅡㅡㅡㅡㅡ")
print(SNSB_all_dataset_torch)
"""

'\nprint("ㅡㅡㅡㅡㅡ[SNSB_all_dataset_torch.shape]ㅡㅡㅡㅡㅡ")\nprint(SNSB_all_dataset_torch.shape)\n\nprint("ㅡㅡㅡㅡㅡ[SNSB_all_dataset_torch]ㅡㅡㅡㅡㅡ")\nprint(SNSB_all_dataset_torch)\n'

In [19]:
# Person_all_dataset_np = Person_all_dataset.to_numpy()
# Person_all_dataset_torch = torch.from_numpy(Person_all_dataset_np).float()

# print("ㅡㅡㅡㅡㅡ[Person_all_dataset_torch.shape]ㅡㅡㅡㅡㅡ")
# print(Person_all_dataset_torch.shape)

# print("ㅡㅡㅡㅡㅡ[Person_all_dataset_torch]ㅡㅡㅡㅡㅡ")
# print(Person_all_dataset_torch)

## - eyerpt, rpt

### 1) 데이터셋 변환

### 2) 데이터셋 불러오기

In [20]:
# 전체 데이터셋 정보
train_eyerpt_all_dataset = defaultdict(list)
test_eyerpt_all_dataset = defaultdict(list)
train_rpt_all_dataset = defaultdict(list)
test_rpt_all_dataset = defaultdict(list)

# 전체 경로
eyerpt_rpt_path = f'C:\\Users\\Bang\\JupyterProjects\\VEEM_Project\\data\\rpt\\'

# 전체 폴더 내 파일 리스트 추출
eyerpt_rpt_files_name = os.listdir(eyerpt_rpt_path)
eyerpt_rpt_files_name = sorted(eyerpt_rpt_files_name)

print("eyerpt_rpt_files_name : ", eyerpt_rpt_files_name)

# eyerpt, rpt 파일 리스트 추출
eyerpt_files_name = [eyerpt_rpt_file_name for eyerpt_rpt_file_name in eyerpt_rpt_files_name if "eye" in eyerpt_rpt_file_name]
rpt_files_name = [eyerpt_rpt_file_name for eyerpt_rpt_file_name in eyerpt_rpt_files_name if not "eye" in eyerpt_rpt_file_name]

print("eyerpt_files_name : ", eyerpt_files_name)
print("rpt_files_name : ", rpt_files_name)

eyerpt_rpt_files_name :  ['02_eyerpt.csv', '02_rpt.csv', '03_eyerpt.csv', '03_rpt.csv', '04_eyerpt.csv', '04_rpt.csv', '06_eyerpt.csv', '06_rpt.csv', '07_eyerpt.csv', '07_rpt.csv', '08_eyerpt.csv', '08_rpt.csv', '09_eyerpt.csv', '09_rpt.csv', '10_eyerpt.csv', '10_rpt.csv', '11_eyerpt.csv', '11_rpt.csv', '12_eyerpt.csv', '12_rpt.csv', '13_eyerpt.csv', '13_rpt.csv', '14_eyerpt.csv', '14_rpt.csv', '15_eyerpt.csv', '15_rpt.csv', '16_eyerpt.csv', '16_rpt.csv', '17_eyerpt.csv', '17_rpt.csv', '18_eyerpt.csv', '18_rpt.csv', '19_eyerpt.csv', '19_rpt.csv', '20_eyerpt.csv', '20_rpt.csv', '21_eyerpt.csv', '21_rpt.csv', '22_eyerpt.csv', '22_rpt.csv', '23_eyerpt.csv', '23_rpt.csv', '24_eyerpt.csv', '24_rpt.csv', '25_eyerpt.csv', '25_rpt.csv', '27_eyerpt.csv', '27_rpt.csv', '28_eyerpt.csv', '28_rpt.csv', '29_eyerpt.csv', '29_rpt.csv', '30_eyerpt.csv', '30_rpt.csv', '31_eyerpt.csv', '31_rpt.csv', '32_eyerpt.csv', '32_rpt.csv', '33_eyerpt.csv', '33_rpt.csv', '34_eyerpt.csv', '34_rpt.csv', '35_eyerpt.cs

### 3) train test split

In [21]:
# 전체 데이터 개수 -> 비율 기반 split
all_eyerpt_count = len(eyerpt_files_name)
test_eyerpt_count = int(all_eyerpt_count * args.test_ratio)
train_eyerpt_count = all_eyerpt_count - test_eyerpt_count

train_eyerpt_files_name = eyerpt_files_name[:train_eyerpt_count]
test_eyerpt_files_name = eyerpt_files_name[train_eyerpt_count:]
"""
print("train_eyerpt_files_name : ", train_eyerpt_files_name)
print("len(train_eyerpt_files_name) : ", len(train_eyerpt_files_name))
print("test_eyerpt_files_name : ", test_eyerpt_files_name)
print("len(test_eyerpt_files_name) : ", len(test_eyerpt_files_name))
"""

all_rpt_count = len(rpt_files_name)
test_rpt_count = int(all_rpt_count * args.test_ratio)
train_rpt_count = all_rpt_count - test_rpt_count

train_rpt_files_name = rpt_files_name[:train_rpt_count]
test_rpt_files_name = rpt_files_name[train_rpt_count:]
"""
print("train_rpt_files_name : ", train_rpt_files_name)
print("len(train_rpt_files_name) : ", len(train_rpt_files_name))
print("test_rpt_files_name : ", test_rpt_files_name)
print("len(test_rpt_files_name) : ", len(test_rpt_files_name))
"""

'\nprint("train_rpt_files_name : ", train_rpt_files_name)\nprint("len(train_rpt_files_name) : ", len(train_rpt_files_name))\nprint("test_rpt_files_name : ", test_rpt_files_name)\nprint("len(test_rpt_files_name) : ", len(test_rpt_files_name))\n'

### 4) 전처리 함수 정의

In [22]:
# eyerpt 파일 전처리
def eyerpt_rpt_preprocessing(name):
    # =====================================================
    # (1) 변수 불러오기
    # =====================================================
    print("name : ", name)
    files_name = eval(f"{name}_files_name")
    all_dataset = eval(f"{name}_all_dataset")

    for i, file_name in enumerate(files_name):
        # =====================================================
        # (2) 데이터 불러오기
        # =====================================================
        path = eyerpt_rpt_path + file_name
        """
        print('path : ', path) # 확인용 코드
        """

        dataset=pd.read_csv(path)
        """
        print("ㅡㅡㅡㅡㅡ[dataset.dtypes]ㅡㅡㅡㅡㅡ")
        print(dataset.dtypes)
        """

        # =====================================================
        # (3) 데이터 자료형 변환
        # =====================================================

        # =====================================================
        # (4) 인덱스 지정
        # =====================================================
        # dataset['time stamp'] = pd.to_datetime(dataset['time stamp'])
        dataset.set_index('time stamp', inplace=True)
        """
        print("ㅡㅡㅡㅡㅡ[dataset]ㅡㅡㅡㅡㅡ")
        print(dataset)
        """

        # =====================================================
        # (5) 데이터 프레임 변환 + 필요한 피쳐 추출
        # =====================================================
        df = dataset.loc[:, :]
        if(name == "train_eyerpt" or name == "test_eyerpt"):
            df = df.drop(['time', 'beforeOBJ', 'presentOBJ', 'Obeject_name'], axis = 1) # 1 = columns
        else:
            df = df.drop(['total_task_time_s', 'hand_x_rotation_deg', 'hand_y_rotation_deg', 'hand_z_rotation_deg'], axis = 1)
        """
        print("ㅡㅡㅡㅡㅡ[df]ㅡㅡㅡㅡㅡ")
        print(df)
        """

        # =====================================================
        # (6) 결측치 행 제거
        # =====================================================
        df.dropna(axis=0, inplace = True)
        """
        print("ㅡㅡㅡㅡㅡ[df]ㅡㅡㅡㅡㅡ")
        print(df)
        print(df.Panel_num)
        """

        # =====================================================
        # (7) 실험 종료 이후 데이터 제거
        # =====================================================
        if(name == "train_eyerpt" or name == "test_eyerpt"):
            df_drop8 = df[df.Panel_num < 8]
        else:
            df_drop8 = df[df.panel_num < 8]
        """
        print("ㅡㅡㅡㅡㅡ[df_drop8]ㅡㅡㅡㅡㅡ")
        print(df_drop8)
        """

        # =====================================================
        # (8) 데이터 프레임 -> numpy 변환 -> torch 변환
        # =====================================================
        if(name == "train_eyerpt" or name == "test_eyerpt"):
            df_drop8_np = df_drop8.to_numpy()
            df_drop8_torch = torch.from_numpy(df_drop8_np)
        else:
            df_drop8_np = df_drop8.to_numpy()
            df_drop8_torch = torch.from_numpy(df_drop8_np).float()
        """
        print("ㅡㅡㅡㅡㅡ[df_drop8_torch.shape]ㅡㅡㅡㅡㅡ")
        print(df_drop8_torch.shape)

        print("ㅡㅡㅡㅡㅡ[df_drop8_torch]ㅡㅡㅡㅡㅡ")
        print(df_drop8_torch)
        """

        # =====================================================
        # (9) 데이터셋 길이 추출
        # =====================================================
        df_sequence_length = len(df_drop8_torch)
        """
        print("df_sequence_length : ", df_sequence_length)
        """
        
        # =====================================================
        # (10) 데이터셋 라벨 행 추출(float형 사용)
        # =====================================================
        if(args.learn_type == "regression"):
            # =====================================================
            # 1] SNSB 데이터셋 라벨 행 추출(float형 사용)
            # =====================================================
            SNSB_label = SNSB_all_dataset_torch[i,:]
            """
            print("SNSB_label : ", SNSB_label)
            """
            SNSB_label = SNSB_label.reshape(-1, len(SNSB_label)) # loss 학습을 위해 output과 형식 통일!!!
            """
            print("SNSB_label : ", SNSB_label)
            """
        else:
            # =====================================================
            # 2] Person_dementia 데이터셋 라벨 행 추출(float형 사용)
            # =====================================================
            Person_dementia_label = Person_dementia_dataset_torch[i]
            """
            print("Person_dementia_label : ", Person_dementia_label)
            """
            Person_dementia_label = Person_dementia_label.reshape(1) # loss 학습을 위해 output과 형식 통일!!!
            """
            print("Person_dementia_label : ", Person_dementia_label)
            """

        # =====================================================
        # (11) 전체 데이터셋 구성
        # =====================================================
        # 파일 정보 + 파일 sequence 길이 리스트화
        if(args.learn_type == "regression"):
            df_infor = [df_drop8_torch, df_sequence_length, SNSB_label]
        else:
            df_infor = [df_drop8_torch, df_sequence_length, Person_dementia_label]
        """
        print("df_infor : ", df_infor)
        """

        # 모든 정보 딕셔너리화
        all_dataset[file_name[0:2]].append(df_infor)
        # *all_dataset = dict(zip([file_name[0:2]], df_infor))
        """
        print("all_dataset['0'] : ", all_dataset['02'])
        print("all_dataset : ", all_dataset)
        """

    # =====================================================
    # (12) 전체 데이터셋 return
    # =====================================================
    exec(f"{name}_all_dataset = all_dataset")

    print("len(all_dataset) : ", len(eval(f"{name}_all_dataset")))

### 5) 전처리

In [23]:
eyerpt_rpt_preprocessing("train_eyerpt")
eyerpt_rpt_preprocessing("test_eyerpt")
eyerpt_rpt_preprocessing("train_rpt")
eyerpt_rpt_preprocessing("test_rpt")

print("len(train_eyerpt_all_dataset) : ", len(train_eyerpt_all_dataset))
print("len(test_eyerpt_all_dataset) : ", len(test_eyerpt_all_dataset))
print("len(train_rpt_all_dataset) : ", len(train_rpt_all_dataset))
print("len(test_rpt_all_dataset) : ", len(test_rpt_all_dataset))

name :  train_eyerpt
len(all_dataset) :  28
name :  test_eyerpt
len(all_dataset) :  9
name :  train_rpt
len(all_dataset) :  28
name :  test_rpt
len(all_dataset) :  9
len(train_eyerpt_all_dataset) :  28
len(test_eyerpt_all_dataset) :  9
len(train_rpt_all_dataset) :  28
len(test_rpt_all_dataset) :  9


### 3) 데이터 자료형 변환

rpt_dataset['panel_num'] = rpt_dataset['panel_num'].astype(float)
rpt_dataset['error'] = rpt_dataset['error'].astype(float)

print("ㅡㅡㅡㅡㅡ[eyerpt_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(eyerpt_dataset.dtypes)
print("ㅡㅡㅡㅡㅡ[rpt_dataset.dtypes]ㅡㅡㅡㅡㅡ")
print(rpt_dataset.dtypes)

## 5. Model Train

### 1) Load model epoch

In [24]:
# Load model
start_epoch = 0
if args.resume:
    start_epoch = load_checkpoint(args.checkpoints, args.title, model, optimizer)

### 2) Train model

In [25]:
for epoch in range(start_epoch, args.epochs):
    """
    # start timer
    start_time = time.time() # 확인용 코드
    """
    # 모델 학습 소요시간
    start_time = time.monotonic()
    
    # train one epoch + evaluate one epoch
    train_summary = train(train_rpt_all_dataset.items(), args.input_size, model, optimizer, scheduler, loss_fn, metric_fn, args.device)

    # write log
    train_logger.add_scalar('Loss', train_summary['loss'], epoch + 1)
    train_logger.add_scalar('Accuracy', train_summary['metric'], epoch + 1)
    
    # save model
    save_checkpoint(args.checkpoints, args.title, model, optimizer, epoch + 1)
        
    # 모델 학습 소요시간
    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Print log
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    if(args.learn_type == "regression"):
        print(f'\t Train Loss: {train_summary["loss"]:.3f}')
    else:
        print(f'\t Train Loss: {train_summary["loss"]:.3f} | Train Acc: {train_summary["metric"]:.2f}%')
    print(f'\t scheduled_lr : {scheduler.get_last_lr()[0]}')

# 모델 저장
torch.save(model.state_dict(), f"{args.title}.ckpt")

Epoch: 01 | Epoch Time: 0m 1s
	 Train Loss: 0.828 | Train Acc: 0.32%
	 scheduled_lr : 0.002999576110003665
Epoch: 02 | Epoch Time: 0m 0s
	 Train Loss: 0.780 | Train Acc: 0.32%
	 scheduled_lr : 0.0029983046795916267
Epoch: 03 | Epoch Time: 0m 0s
	 Train Loss: 0.749 | Train Acc: 0.32%
	 scheduled_lr : 0.0029961864273593984
Epoch: 04 | Epoch Time: 0m 0s
	 Train Loss: 0.724 | Train Acc: 0.32%
	 scheduled_lr : 0.0029932225505148844
Epoch: 05 | Epoch Time: 0m 0s
	 Train Loss: 0.707 | Train Acc: 0.32%
	 scheduled_lr : 0.00298941472420175
Epoch: 06 | Epoch Time: 0m 0s
	 Train Loss: 0.690 | Train Acc: 0.54%
	 scheduled_lr : 0.002984765100552636
Epoch: 07 | Epoch Time: 0m 0s
	 Train Loss: 0.679 | Train Acc: 0.68%
	 scheduled_lr : 0.0029792763074728093
Epoch: 08 | Epoch Time: 0m 0s
	 Train Loss: 0.667 | Train Acc: 0.71%
	 scheduled_lr : 0.0029729514471549113
Epoch: 09 | Epoch Time: 0m 0s
	 Train Loss: 0.655 | Train Acc: 0.71%
	 scheduled_lr : 0.0029657940943256275
Epoch: 10 | Epoch Time: 0m 0s
	 

Epoch: 78 | Epoch Time: 0m 0s
	 Train Loss: 0.388 | Train Acc: 0.86%
	 scheduled_lr : 0.001080290443997843
Epoch: 79 | Epoch Time: 0m 0s
	 Train Loss: 0.387 | Train Acc: 0.86%
	 scheduled_lr : 0.0010461753938595707
Epoch: 80 | Epoch Time: 0m 0s
	 Train Loss: 0.387 | Train Acc: 0.86%
	 scheduled_lr : 0.001012316839335477
Epoch: 81 | Epoch Time: 0m 0s
	 Train Loss: 0.386 | Train Acc: 0.86%
	 scheduled_lr : 0.0009787339168289663
Epoch: 82 | Epoch Time: 0m 0s
	 Train Loss: 0.386 | Train Acc: 0.86%
	 scheduled_lr : 0.0009454456069599035
Epoch: 83 | Epoch Time: 0m 0s
	 Train Loss: 0.386 | Train Acc: 0.86%
	 scheduled_lr : 0.0009124707238370192
Epoch: 84 | Epoch Time: 0m 0s
	 Train Loss: 0.385 | Train Acc: 0.86%
	 scheduled_lr : 0.0008798279044244285
Epoch: 85 | Epoch Time: 0m 0s
	 Train Loss: 0.385 | Train Acc: 0.86%
	 scheduled_lr : 0.0008475355980082671
Epoch: 86 | Epoch Time: 0m 0s
	 Train Loss: 0.384 | Train Acc: 0.86%
	 scheduled_lr : 0.0008156120557693982
Epoch: 87 | Epoch Time: 0m 0s


## 6. Model Test

In [26]:
# 학습된 모델 불러오기
model.load_state_dict(torch.load(f"{args.title}.ckpt"))

# 모델 성능 측정
test_summary = evaluate(test_rpt_all_dataset.items(), args.input_size, model, loss_fn, metric_fn, args.device)

# write log
test_logger.add_scalar('Loss', test_summary['loss'], epoch + 1)
test_logger.add_scalar('Accuracy', test_summary['metric'], epoch + 1)

if(args.learn_type == "regression"):
    print(f'\t Test Loss: {test_summary["loss"]:.3f}')
else:
    print(f'\t Test Loss: {test_summary["loss"]:.3f} | Test Acc: {test_summary["metric"]:.2f}%')

	 Test Loss: 0.423 | Test Acc: 0.78%
